In [1]:
! pip install mysql-connector-python

In [2]:
import mysql.connector
import pandas as pd
import mysql.connector


In [3]:
conn=mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="daproject2"
)

In [4]:
conn.is_connected()

True

In [5]:
#we will create an function out it

def connect_db():
    return mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="daproject2")

In [6]:
connect_db().is_connected()

True

In [7]:
db=connect_db()
cursor=db.cursor()

In [8]:
"select count(*) as total_soppliers from suppliers"

'select count(*) as total_soppliers from suppliers'

In [9]:
cursor.execute("select count(*) as total_suppliers from suppliers")

In [10]:
row=cursor.fetchone()


In [11]:
cursor.execute("SHOW TABLES")
print(cursor.fetchall())

[('products',), ('reorders',), ('shipments',), ('stock_entries',), ('suppliers',)]


In [12]:
# Re-fetch the suppliers count
cursor.execute("select count(*) as total_suppliers from suppliers")
row = cursor.fetchone()
print("Total suppliers:", row[0])

Total suppliers: 51


In [13]:
queries = {
"Total Suppliers": "SELECT COUNT(*) AS count FROM suppliers",

"Total Products": "SELECT COUNT(*) AS count FROM products",

"Total Categories Dealing": "SELECT COUNT(DISTINCT category) AS count FROM products",

"Total Sale Value (Last 3 Months)": """
SELECT ROUND(SUM(ABS(se.change_quantity) * p.price), 2) AS total_sale
FROM stock_entries se
JOIN products p ON se.product_id = p.product_id
WHERE se.change_type = 'Sale'
AND se.entry_date >= (
SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
""",

"Total Restock Value (Last 3 Months)": """
SELECT ROUND(SUM(se.change_quantity * p.price), 2) AS total_restock
FROM stock_entries se
JOIN products p ON se.product_id = p.product_id
WHERE se.change_type = 'Restock'
AND se.entry_date >= (
SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
""",

"Below Reorder & No Pending Reorders": """
SELECT COUNT(*) AS below_reorder
FROM products p
WHERE p.stock_quantity < p.reorder_level
AND p.product_id NOT IN (
SELECT DISTINCT product_id FROM reorders WHERE status = 'Pending')
"""
}

In [14]:
result={}
for label,query in queries.items():
    cursor.execute(query)
    row=cursor.fetchone()
    result[label]=row[0]

In [15]:
result

{'Total Suppliers': 51,
 'Total Products': 201,
 'Total Categories Dealing': 6,
 'Total Sale Value (Last 3 Months)': Decimal('1529237.11'),
 'Total Restock Value (Last 3 Months)': Decimal('6690663.49'),
 'Below Reorder & No Pending Reorders': 14}

In [16]:

## Now lets create function out of it 

def get_basic_info(cursor):
    """
    Retrieve summary inventory and supply chain metrics.

    Args:
        cursor (mysql.connector.cursor.MySQLCursorDict): Cursor object with dictionary=True.

    Returns:
        dict: Dictionary of metric labels and their values.
    """

    queries = {
        "Total Suppliers": "SELECT COUNT(*) AS count FROM suppliers",

        "Total Products": "SELECT COUNT(*) AS count FROM products",

        "Total Categories Dealing": "SELECT COUNT(DISTINCT category) AS count FROM products",

        "Total Sale Value (Last 3 Months)": """
            SELECT ROUND(SUM(ABS(se.change_quantity) * p.price), 2) AS total_sale
            FROM stock_entries se
            JOIN products p ON se.product_id = p.product_id
            WHERE se.change_type = 'Sale'
              AND se.entry_date >= (
                  SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
        """,

        "Total Restock Value (Last 3 Months)": """
            SELECT ROUND(SUM(se.change_quantity * p.price), 2) AS total_restock
            FROM stock_entries se
            JOIN products p ON se.product_id = p.product_id
            WHERE se.change_type = 'Restock'
              AND se.entry_date >= (
                  SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
        """,

        "Below Reorder & No Pending Reorders": """
            SELECT COUNT(*) AS below_reorder
            FROM products p
            WHERE p.stock_quantity < p.reorder_level
              AND p.product_id NOT IN (
                  SELECT DISTINCT product_id FROM reorders WHERE status = 'Pending')
        """
    }

    results = {}
    for label, query in queries.items():
        cursor.execute(query)
        row = cursor.fetchone()
        # Since row is a dictionary, extract the single value by getting the first value in dict.values()
        results[label] = list(row.values())[0]

    return results





In [17]:
def get_additonal_tables(cursor):
    queries = {
        "Suppliers Contact Details": "SELECT supplier_name, contact_name, email, phone FROM suppliers",

        "Products with Supplier and Stock": """
            SELECT 
                p.product_name,
                s.supplier_name,
                p.stock_quantity,
                p.reorder_level
            FROM products p
            JOIN suppliers s ON p.supplier_id = s.supplier_id
            ORDER BY p.product_name ASC
        """,

        "Products Needing Reorder": """
            SELECT product_name, stock_quantity, reorder_level
            FROM products
            WHERE stock_quantity <= reorder_level
        """
    }

    tables = {}
    for label, query in queries.items():
        cursor.execute(query)
        tables[label] = cursor.fetchall()

    return tables

In [18]:
get_additonal_tables(cursor)

{'Suppliers Contact Details': [('supplier_name',
   'contact_name',
   'email',
   'phone'),
  ('Anderson-Thompson',
   'Bonnie Davis',
   'zacharysanchez@hotmail.com',
   '829.485.9853x0522'),
  ('Rowland Ltd', 'Beth Stevens', 'joshua60@yahoo.com', '(779)942-0726'),
  ('Baxter-Meadows',
   'Lisa Lewis',
   'andersonchristina@yahoo.com',
   '449-766-7325'),
  ('Wilson, Graham and Williams',
   'David Martinez',
   'abrown@hotmail.com',
   '+1-653-827-5215x266'),
  ('Smith, Kennedy and Moreno',
   'Victoria Gonzalez',
   'mary30@williams-moore.com',
   '891-859-2775x35297'),
  ('Middleton LLC',
   'Megan Miller',
   'kelseywilliams@gmail.com',
   '001-724-731-6199x4596'),
  ('Evans Inc',
   'Danielle Moore',
   'ihoffman@warren.com',
   '+1-951-447-1975x770'),
  ('Lawrence, Garcia and Hernandez',
   'Tamara Johnson',
   'georgeherrera@hotmail.com',
   '+1-635-460-8476x270'),
  ('Young, Browning and Ware',
   'Heather Hill',
   'stephensjason@yahoo.com',
   '(514)361-6411x489'),
  ('Newt